In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import seaborn as sns
import os
import configs
from math import floor, ceil

from Model import Model
from PlaceInputs import PlaceInputs
from utils import *
from utils_summary_statistics import *
import configs

# Choose experiment to load

In [2]:
# Specify what you're looking at
exp = 'narrow_search_factor'
model_type = 'place_field_ablation'

In [3]:
if os.environ['USER'] == 'chingfang':
    engram_dir = '/Volumes/aronov-locker/Ching/barcodes/' # Local Path
elif 'SLURM_JOBID' in os.environ.keys():
    engram_dir = '/mnt/smb/locker/aronov-locker/Ching/barcodes/' # Axon Path
else:
    engram_dir = '/home/cf2794/engram/Ching/barcodes/' # Cortex Path
exp_dir = os.path.join(engram_dir, 'resolution', exp, model_type)

In [4]:
N_inp = 5000
N_bar = 5000
num_states = 100
inputs = PlaceInputs(N_inp, num_states).get_inputs()

In [5]:
truncate = False

In [6]:
summary_stats = {
    'identification_1': None, 'identification_2': None,
    'identification_3': None, 'identification_4': None,
    'reconstruct_1': None, 'reconstruct_2': None,
    'reconstruct_3': None, 'activations_1': None, 'activations_2': None
    }

for param in os.listdir(exp_dir):
    if param == '.DS_Store': continue
    dirpath = os.path.join(exp_dir, param)
    for site_spacing in os.listdir(dirpath):
        if not site_spacing.startswith('res'): continue    
        dirpath = os.path.join(exp_dir, param, site_spacing)
        for seed in os.listdir(dirpath):
            if not seed.startswith('seed'): continue
            dirpath = os.path.join(exp_dir, param, site_spacing, seed)
            _param = round(float(param), 1)
            _seed = int(seed[4:])
            _site_spacing = int(site_spacing[3:])
            if truncate and (_seed > 5 or _param > 0.6): continue
            cache_states = [0, _site_spacing, 66]
        
            try:
                with open(os.path.join(dirpath, 'results.p'), 'rb') as f:
                    _results = pickle.load(f)
            except:
                continue
                
            readout = np.linalg.norm(_results['narrow_reconstruct'], axis=1)
            readout /= readout.max()
            reconstruct = _results['narrow_reconstruct']
            activations = _results['narrow_acts'].copy()
            _summary_stats = get_resolution_summary_statistics(
                readout, reconstruct, cache_states, activations,
                inputs, _site_spacing, search_strength=_param)
            for key, _dict in _summary_stats.items():
                n_samples = len(_dict[list(_dict.keys())[0]])
                if summary_stats[key] is None:
                    summary_stats[key] = _dict
                else:
                    for nested_key in _dict.keys():
                        summary_stats[key][nested_key].extend(_dict[nested_key])

/share/ctn/users/cf2794/Code/barcodes/utils_summary_statistics.py:132: RuntimeWarning: invalid value encountered in greater
  valid = np.logical_and(readout>0.5, np.isin(peak_locs, cache_states))
